<a href="https://colab.research.google.com/github/tobyloby12/AMLS-assignment/blob/main/Task%20B/transfer_learning_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# mounting google drive (not needed if not on google colab)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# put correct path to AMLSassignment github folder
path = '/content/drive/MyDrive/amlsAssignment'

In [ ]:
import os
os.chdir(path)

In [ ]:
import pandas as pd
import os

In [ ]:
# using the image_dataset_from_directory function to create the datasets for training of neural networks from the directory created above

from tensorflow.keras.preprocessing import image_dataset_from_directory
os.chdir(path)
train_dataset = image_dataset_from_directory(
    os.getcwd() + '/dataset/class_dataset',
    validation_split=0.3,
    subset='training',
    seed=0,
    image_size=(256, 256),
    batch_size=32

)

validate_dataset = image_dataset_from_directory(
    os.getcwd() + '/dataset/class_dataset',
    validation_split=0.3,
    subset='validation',
    seed=0,
    image_size=(256, 256),
    batch_size=32
)

test_dataset = image_dataset_from_directory(
    os.getcwd() + '/dataset/class_dataset_test',
    seed=0,
    image_size=(256, 256),
    batch_size=32
)

Found 3000 files belonging to 4 classes.
Using 2100 files for training.
Found 3000 files belonging to 4 classes.
Using 900 files for validation.
Found 200 files belonging to 4 classes.


In [ ]:
# preparing datasets like in the transfer learning augmented notebook
import tensorflow as tf
from tensorflow.keras import layers
IMG_SIZE = 256

resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(IMG_SIZE, IMG_SIZE),
  layers.Rescaling(1./255)
])

data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.5),
])

batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def prepare(ds, shuffle=False, augment=False):
  # Resize and rescale all datasets.
  ds = ds.map(lambda x, y: (resize_and_rescale(x), y), 
              num_parallel_calls=AUTOTUNE)

  if shuffle:
    ds = ds.shuffle(1000)

  # Batch all datasets.
  # ds = ds.batch(batch_size)

  # Use data augmentation only on the training set.
  if augment:
    ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), 
                num_parallel_calls=AUTOTUNE)
    # ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), 
    #             num_parallel_calls=AUTOTUNE)


  # Use buffered prefetching on all datasets.
  return ds.prefetch(buffer_size=AUTOTUNE)

ds_train = prepare(train_dataset, augment=True, shuffle=True)
ds_validate = prepare(validate_dataset)
ds_test = prepare(test_dataset)

In [ ]:
# creating function that will produce a classification report and append it to a dataframe so that the model accuracies and f1 scores can be comnpared
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report
from keras.applications.vgg16 import VGG16
from keras.models import Model
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras import models
def evaluate(model_name, df, model_weights, augment=False):
  # if function to choose whether data is augmented or not
  if augment == False:
    data = test_dataset
  else:
    data = ds_test
  # if else clause to choose what model is being used
  if model_name == 'VGG16':
    model = VGG16(weights="imagenet", include_top=False, input_shape=(256, 256, 3))
    model = models.Sequential([
      model,
      Flatten(),
      Dense(1024, activation='relu'),
      Dense(4, activation='softmax')
    ])


  elif model_name == 'Xception':
      xception_model = tf.keras.applications.Xception(
          include_top=False,
          weights="imagenet",
          input_tensor=None,
          input_shape=(256, 256, 3),
          pooling=None
      )
      model = models.Sequential([
          xception_model,
          Flatten(),
          Dense(1024, activation='relu'),
          Dense(4, activation='softmax')
      ])


  elif model_name == 'efficientNet':
    efficient_model = tf.keras.applications.EfficientNetB4(
        include_top=False,
        weights="imagenet",
        input_tensor=None,
        input_shape=(256, 256, 3),
        pooling=True,
        classes=1000,
        classifier_activation="softmax"
    )
    model = models.Sequential([
          efficient_model,
          Flatten(),
          Dense(10, activation='relu'),
          Dense(4, activation='softmax')
    ])

  # loading model and making predictions
  model.load_weights(model_weights)
  predictions = np.array([])
  labels =  np.array([])
  for x, y in data:
    predictions = np.concatenate([predictions, np.argmax(model.predict(x), axis = -1)])
    labels = np.concatenate([labels, y])

  # getting classification report to append to dataframe
  report = classification_report(
      labels,
      predictions, 
      output_dict=True, 
      target_names=['meningioma_tumor', 
                    'no_tumor', 
                    'glioma_tumor', 
                    'pituitary_tumor'])
  
  row = {
      'Architecture': model_name,
      'Accuracy': "{:.4f}".format(report['accuracy']), 
      'F1 meningioma': "{:.4f}".format(report['meningioma_tumor']['f1-score']), 
      'F1 no_tumor': "{:.4f}".format(report['no_tumor']['f1-score']), 
      'F1 glioma_tumor': "{:.4f}".format(report['glioma_tumor']['f1-score']), 
      'F1 pituitary_tumor': "{:.4f}".format(report['pituitary_tumor']['f1-score'])
  }
  # appending the row to datafrome with all the information
  df = df.append(row, ignore_index=True)
    
  return df

In [ ]:
# creating initial dataframe
df = pd.DataFrame(columns=['Architecture', 'Accuracy', 'F1 meningioma', 'F1 no_tumor', 'F1 glioma_tumor', 'F1 pituitary_tumor'])

In [ ]:
# evaluating all models
df = evaluate('VGG16', df, 'best_model_VGG16_3.h5')
df = evaluate('Xception', df, 'best_model_Xception_1.h5')
df = evaluate('efficientNet', df, 'best_model_efficient_4.h5')
df = evaluate('VGG16', df, 'best_model_VGG16_augment_1.h5', augment=True)
df = evaluate('Xception', df, 'best_model_Xception_augmented_1.h5', augment=True)
df = evaluate('efficientNet', df, 'best_model_efficient_augmented_2.h5', augment=True)

71696384/71686520 [==============================] - 1s 0us/step


In [ ]:
# saving dataframe
df.to_csv('results_CNN.csv')